In [ ]:
import sys
import stereo as st
import pandas as pd
import numpy as np

In [ ]:
# read data
data1 = st.io.read_gef('./B03208C214.tissue.gef', bin_size=20)
data2 = st.io.read_gef('./B03208F213.tissue.gef', bin_size=20)
data3 = st.io.read_gef('./B03208G312.tissue.gef', bin_size=20)
data4 = st.io.read_gef('./B03208A213.tissue.gef', bin_size=20)


# Preprocessing

In [ ]:
data1.tl.cal_qc()
data2.tl.cal_qc()
data3.tl.cal_qc()
data4.tl.cal_qc()

# plot before filtering

In [ ]:
data1.plt.genes_count()

In [ ]:
data1.plt.violin()

In [ ]:
data2.plt.genes_count()

In [ ]:
data2.plt.violin()

In [ ]:
data3.plt.genes_count()

In [ ]:
data3.plt.violin()

In [ ]:
data4.plt.genes_count()

In [ ]:
data4.plt.violin()

In [ ]:
data1.tl.raw_checkpoint()
data2.tl.raw_checkpoint()
data3.tl.raw_checkpoint()
data4.tl.raw_checkpoint()

In [ ]:
# filter cells
data1.tl.filter_cells(min_n_genes_by_counts=10, max_n_genes_by_counts=600, 
                      min_total_counts=200, max_total_counts=1500, 
                     pct_counts_mt=10, inplace=True)
data2.tl.filter_cells(min_n_genes=5, max_n_genes_by_counts=500, 
                      min_total_counts=200, max_total_counts=1500, 
                      pct_counts_mt=10, inplace=True)
data3.tl.filter_cells(min_n_genes_by_counts=5, max_n_genes_by_counts=600, 
                     min_total_counts=200, max_total_counts=1500, 
                      pct_counts_mt=5, inplace=True)
data4.tl.filter_cells(min_n_genes=5, max_n_genes_by_counts=500, 
                      min_total_counts=200, max_total_counts=1500, 
                      pct_counts_mt=10, inplace=True)

# Processing on MSData

In [ ]:
from stereo.core.ms_data import MSData
from stereo.core.ms_pipeline import slice_generator

In [ ]:
# pass name list to MSData
name_list=['control_1','control_2','mutant_1','mutant_2']
name_list

In [ ]:
ms_data = MSData(_data_list=[data1, data2, data3, data4])

In [ ]:
ms_data

In [ ]:
ms_data.names=name_list
ms_data

# Normalization

In [ ]:
# save raw data beforehand
ms_data.tl.raw_checkpoint()

In [ ]:
# ms_data.integrate() is necessarily to be performed after data loading. Default method is intersect, which means to take the intersection of genes (var) for subsequent multi-sample analysis.
ms_data.integrate()
ms_data

In [ ]:
ms_data.tl.cal_qc(scope=slice_generator[:],mode='integrate')


In [ ]:
ms_data.plt.violin()

In [ ]:
ms_data.plt.spatial_scatter(
            scope=slice_generator[:],
            mode='integrate',
            plotting_scale_width=10,          # the width of scale
            reorganize_coordinate=2,          # the number of plots in each row
            horizontal_offset_additional=20,  # adjustment for horizontal distance
            vertical_offset_additional=20     # adjustment for vertical distance
            )

In [ ]:
ms_data.tl.normalize_total(scope=slice_generator[:], mode='integrate')
ms_data.tl.log1p(scope=slice_generator[:], mode='integrate')

In [ ]:
#Highly variable genes
ms_data.tl.highly_variable_genes(scope=slice_generator[:],mode='integrate',min_mean=0.0125, max_mean=3,min_disp=0.5, n_top_genes=2000, res_key='highly_variable_genes')
ms_data.tl.scale(scope=slice_generator[:],mode='integrate', zero_center=False, max_value=10)

In [ ]:
st.io.write_h5ms(ms_data,output='./integrate_log1p_b4PCA.h5ms')

In [ ]:
ms_data.tl.spatial_hotspot(
                    use_highly_genes=True,
                    use_raw=True,
                    hvg_res_key='highly_variable_genes',
                    model='normal',
                    n_neighbors=30,
                    n_jobs=20,
                    fdr_threshold=0.05,
                    min_gene_threshold=10,
                    res_key='spatial_hotspot',
                    )

In [ ]:
st.io.write_h5ms(ms_data,output='./integrate_log1p_b4PCA.h5ms')

In [ ]:
# load h5ms object
ms_data = st.io.read_h5ms('./integrate_log1p_b4PCA.h5ms')

In [ ]:
#Spatial hotspot is a tool for identifying informative genes or gene modules in a single-cell dataset. Importantly, ‘informative’ here is defined based on how well a gene’s variation agrees with certain cell metric - some similarity mapping between cells.
#Genes which are informative are those whose expression varies in similar way among cells which are nearby in the given metric.

ms_data.plt.hotspot_local_correlations()

In [ ]:
ms_data.plt.hotspot_modules()

# PCA

In [ ]:
ms_data.tl.pca(scope=slice_generator[:], mode='integrate', use_highly_genes=False, n_pcs=30, res_key='pca')

# Batch effect

In [ ]:
from stereo.algorithm.batch_qc.main import BatchQc

In [ ]:
# embedding
ms_data.tl.neighbors(scope=slice_generator[:],mode='integrate', pca_res_key='pca', res_key='neighbors', n_jobs=-1)
ms_data.tl.umap(scope=slice_generator[:],mode='integrate', pca_res_key='pca', neighbors_res_key='neighbors', res_key='umap')

In [ ]:
# clustering
ms_data.tl.leiden(scope=slice_generator[:],mode='integrate', neighbors_res_key='neighbors', res_key='leiden')

In [ ]:
# Batch evaluation
ms_data.tl.batch_qc(scope=slice_generator[:],mode='integrate', cluster_res_key='leiden', report_path='./batch_qc', res_key='batch_qc')

In [ ]:
# Display report
ms_data.plt.show_batch_qc_report(scope=slice_generator[:], mode='integrate', res_key='batch_qc')

# Integrating with batch correction using Harmony

In [ ]:
ms_data.tl.batches_integrate(scope=slice_generator[:], mode='integrate', pca_res_key='pca', res_key='pca_integrated')

# UMAP

In [ ]:
ms_data.tl.neighbors(scope=slice_generator[:], mode='integrate', pca_res_key='pca_integrated', n_pcs=30, res_key='neighbors_integrated', n_jobs=-1)
ms_data.tl.umap(scope=slice_generator[:], mode='integrate', pca_res_key='pca_integrated', neighbors_res_key='neighbors_integrated', res_key='umap_integrated')
ms_data.plt.batches_umap(scope=slice_generator[:], mode='integrate', res_key='umap_integrated')

# Clustering

In [ ]:
ms_data.tl.leiden(scope=slice_generator[:], mode='integrate', neighbors_res_key='neighbors_integrated',
                  resolution=0.4, res_key='leiden0.4')
#ms_data.tl.leiden(scope=slice_generator[:], mode='integrate', neighbors_res_key='umap_integrated',
#                  resolution=1.0, res_key='leiden1.0')

In [ ]:
ms_data.plt.cluster_scatter(
            res_key='leiden0.4',
            scope=slice_generator[:],
            mode='integrate',
            plotting_scale_width=10,          # the width of scale
            reorganize_coordinate=2,          # the number of plots in each row
            horizontal_offset_additional=20,  # adjustment for horizontal distance
            vertical_offset_additional=20     # adjustment for vertical distance
            )

In [ ]:
ms_data.plt.cluster_scatter(
            res_key='leiden0.4',
            scope=slice_generator[:],
            mode='integrate',
            plotting_scale_width=10,          # the width of scale
            reorganize_coordinate=2,          # the number of plots in each row
            horizontal_offset_additional=20,  # adjustment for horizontal distance
            vertical_offset_additional=20,     # adjustment for vertical distance
            groups=['9', '10'])

In [ ]:
ms_data.plt.interact_cluster(res_key='leiden0.4',
                             scope=slice_generator[:],
            mode='integrate',
            #plotting_scale_width=10,          # the width of scale
            reorganize_coordinate=2,          # the number of plots in each row
            horizontal_offset_additional=20,  # adjustment for horizontal distance
            vertical_offset_additional=20)     # adjustment for vertical distance)

In [ ]:
ms_data.plt.umap(
            cluster_key='leiden0.4',
            res_key='umap_integrated',
            scope=slice_generator[:],
            mode='integrate'
            )

# Save output file as h5ms

In [ ]:
st.io.write_h5ms(ms_data,output='./integrate_res0.4_log1p.h5ms')

In [ ]:
# load h5ms object
ms_data = st.io.read_h5ms('./integrate_res0.4_log1p.h5ms')

# Find Marker Genes

In [ ]:
ms_data.tl.find_marker_genes(cluster_res_key='leiden0.4', use_highly_genes=False, use_raw =False, method = 't_test', sort_by = 'scores',
                            res_key='marker_genes_{}'.format('leiden0.4'.replace('leiden', 'ld')), output='all_markers_lasso_res0.4.csv')
#ms_data.tl.filter_marker_genes(marker_genes_res_key='marker_genes_{}'.format('leiden0.4'.replace('leiden', 'ld')), 
#                              min_fold_change=0.25, #min_in_group_fraction=0.25, #max_out_group_fraction=0.5,
#                              compare_abs=False, res_key='marker_genes_filtered_{}'.format(cls_key.replace('leiden', 'ld')),
#                              output='all_markers_filtered_FC0.25_res0.4.csv')

In [ ]:
print( ms_data.merged_data.tl.result['marker_genes_ld0.4'].keys())
print( ms_data.merged_data.tl.result['marker_genes_ld0.4']['1.vs.rest'].head())
print( ms_data.merged_data.tl.result['marker_genes_ld0.4']['1.vs.rest'].isna().sum())

In [ ]:
ms_data.plt.marker_genes_heatmap(res_key = 'marker_genes_ld0.4')

# Annotation with cell labels

In [ ]:
# read data
ref_file = 'MouseRNAseqData.h5ad'
ref = st.io.read_h5ad(ref_file)

In [ ]:
ms_data.tl.single_r(
    ref_exp_data=ref,
    ref_use_col='label.main',
    res_key='annotation',
    fine_tune_times=1,
    n_jobs=40
)

In [ ]:
ms_data.plt.cluster_scatter(res_key='annotation')

In [ ]:
st.io.write_h5ms(ms_data,output='./integrate_lasso_res0.4_annotation.h5ms')

# DEG analysis

In [ ]:
# load h5ms object
ms_data = st.io.read_h5ms('./integrate_res0.4_log1p.h5ms')
ms_data

In [ ]:
# Ensure that leiden clustering results exist
print(type(ms_data.tl.result['leiden0.4']))
print(ms_data.tl.result['leiden0.4'].head())  # Inspect the first few rows

In [ ]:
print("Available Leiden clusters:", ms_data.tl.result['leiden0.4']['group'].unique())

In [ ]:
ms_data

In [ ]:
# Define sample groups
control_samples = ['0','1']
mutant_samples = ['2','3']

In [ ]:
# Create a metadata label for sample groups
sample_labels = ms_data.obs['batch'].copy()
print(sample_labels)

In [ ]:
sample_labels = sample_labels.replace(control_samples, 'control').replace(mutant_samples, 'mutant')
print(sample_labels)

In [ ]:
# Add the sample group label to ms_data
ms_data.obs['condition'] = sample_labels

In [ ]:
print(ms_data.obs['condition'])

In [ ]:
ms_data.obs['condition_cluster'] = ms_data.obs['condition'].astype(str) + '_' + ms_data.obs['leiden0.4'].astype(str)

In [ ]:
print(ms_data.obs['condition_cluster'])

In [ ]:
# Perform Differential Expression Analysis (DEG), repeat for each cluster
ms_data.tl.find_marker_genes(
    cluster_res_key='condition_cluster',  # Compare "control" vs "mutant"
    method='t_test',            # Use t test
    case_groups=['mutant_1'],   # replace the number for other clusters
    control_groups=['control_1'], # replace the number for other clusters
    use_raw=True, res_key='DEGs', 
    output='DEGs_ctrl_vs_mut_cluster1_res0.4.csv',  # replace the number for other clusters
)

In [ ]:
#print( ms_data.merged_data.tl.result['DEGs'].keys())
#print( ms_data.merged_data.tl.result['DEGs']['mutant_1.vs.control_1'].head())
#print( ms_data.merged_data.tl.result['DEGs']['mutant_1.vs.control_1'].isna().sum())